# VisDrone Dataset Exploration

This notebook explores the VisDrone dataset to understand its structure, characteristics, and suitability for autonomous drone detection tasks.

## Table of Contents
1. [Dataset Overview](#dataset-overview)
2. [Data Loading and Structure](#data-loading)
3. [Class Distribution Analysis](#class-distribution)
4. [Image Characteristics](#image-characteristics)
5. [Annotation Analysis](#annotation-analysis)
6. [Challenges and Opportunities](#challenges)

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from pathlib import Path
from collections import defaultdict, Counter
import json

# Add project root to path
sys.path.append('..')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuration
DATASET_DIR = Path('../data/visdrone')
FIGURES_DIR = Path('../results/figures')
FIGURES_DIR.mkdir(exist_ok=True)

print("Environment setup complete!")

Environment setup complete!


## Dataset Overview {#dataset-overview}

The VisDrone dataset is a large-scale benchmark dataset for object detection and tracking in drone-captured images.

In [3]:
# VisDrone class definitions
VISDRONE_CLASSES = {
    0: 'ignored',
    1: 'pedestrian',
    2: 'people',
    3: 'bicycle',
    4: 'car',
    5: 'van',
    6: 'truck',
    7: 'tricycle',
    8: 'awning-tricycle',
    9: 'bus',
    10: 'motor'
}

# Check dataset structure
def explore_dataset_structure(dataset_dir):
    """Explore the structure of the VisDrone dataset."""
    structure = {}
    
    for split in ['train', 'val', 'test']:
        split_dir = dataset_dir / split
        if split_dir.exists():
            images_dir = split_dir / 'images'
            annotations_dir = split_dir / 'annotations'
            
            structure[split] = {
                'exists': True,
                'images': len(list(images_dir.glob('*.jpg'))) if images_dir.exists() else 0,
                'annotations': len(list(annotations_dir.glob('*.txt'))) if annotations_dir.exists() else 0
            }
        else:
            structure[split] = {'exists': False}
    
    return structure

dataset_structure = explore_dataset_structure(DATASET_DIR)
print("Dataset Structure:")
for split, info in dataset_structure.items():
    if info['exists']:
        print(f"  {split}: {info['images']} images, {info['annotations']} annotations")
    else:
        print(f"  {split}: Not found")

Dataset Structure:
  train: Not found
  val: Not found
  test: Not found


## Data Loading and Structure {#data-loading}

In [ ]:
def load_visdrone_annotations(annotation_file):
    """Load VisDrone annotations from a text file."""
    annotations = []
    
    if not annotation_file.exists():
        return annotations
    
    with open(annotation_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            parts = line.split(',')
            if len(parts) != 8:
                continue
            
            try:
                annotation = {
                    'bbox_left': int(parts[0]),
                    'bbox_top': int(parts[1]),
                    'bbox_width': int(parts[2]),
                    'bbox_height': int(parts[3]),
                    'score': int(parts[4]),
                    'object_category': int(parts[5]),
                    'truncation': int(parts[6]),
                    'occlusion': int(parts[7])
                }
                annotations.append(annotation)
            except ValueError:
                continue
    
    return annotations

def analyze_split(dataset_dir, split_name, max_files=None):
    """Analyze a dataset split."""
    split_dir = dataset_dir / split_name
    if not split_dir.exists():
        return None
    
    images_dir = split_dir / 'images'
    annotations_dir = split_dir / 'annotations'
    
    image_files = list(images_dir.glob('*.jpg'))
    if max_files:
        image_files = image_files[:max_files]
    
    analysis = {
        'split': split_name,
        'total_images': len(image_files),
        'image_sizes': [],
        'annotations_per_image': [],
        'class_distribution': Counter(),
        'bbox_areas': [],
        'truncation_stats': Counter(),
        'occlusion_stats': Counter()
    }
    
    for img_file in image_files:
        # Load image to get dimensions
        img = cv2.imread(str(img_file))
        if img is not None:
            h, w = img.shape[:2]
            analysis['image_sizes'].append((w, h))
        
        # Load corresponding annotation
        ann_file = annotations_dir / (img_file.stem + '.txt')
        annotations = load_visdrone_annotations(ann_file)
        
        analysis['annotations_per_image'].append(len(annotations))
        
        for ann in annotations:
            analysis['class_distribution'][ann['object_category']] += 1
            analysis['bbox_areas'].append(ann['bbox_width'] * ann['bbox_height'])
            analysis['truncation_stats'][ann['truncation']] += 1
            analysis['occlusion_stats'][ann['occlusion']] += 1
    
    return analysis

# Analyze train split (sample for speed)
print("Analyzing dataset splits...")
train_analysis = analyze_split(DATASET_DIR, 'train', max_files=1000)
val_analysis = analyze_split(DATASET_DIR, 'val')

print(f"Train analysis: {train_analysis['total_images'] if train_analysis else 0} images processed")
print(f"Validation analysis: {val_analysis['total_images'] if val_analysis else 0} images processed")

## Class Distribution Analysis {#class-distribution}

In [ ]:
def plot_class_distribution(analysis_results, save_path=None):
    """Plot class distribution across dataset splits."""
    fig, axes = plt.subplots(1, len(analysis_results), figsize=(15, 6))
    if len(analysis_results) == 1:
        axes = [axes]
    
    for i, (split_name, analysis) in enumerate(analysis_results.items()):
        if analysis is None:
            continue
            
        # Prepare data
        classes = []
        counts = []
        
        for class_id in sorted(analysis['class_distribution'].keys()):
            if class_id in VISDRONE_CLASSES:
                classes.append(VISDRONE_CLASSES[class_id])
                counts.append(analysis['class_distribution'][class_id])
        
        # Create bar plot
        ax = axes[i]
        bars = ax.bar(range(len(classes)), counts)
        ax.set_title(f'{split_name.title()} Split Class Distribution')
        ax.set_xlabel('Object Class')
        ax.set_ylabel('Number of Instances')
        ax.set_xticks(range(len(classes)))
        ax.set_xticklabels(classes, rotation=45, ha='right')
        
        # Add value labels on bars
        for bar, count in zip(bars, counts):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(counts)*0.01,
                   str(count), ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

# Plot class distributions
analysis_dict = {}
if train_analysis:
    analysis_dict['train'] = train_analysis
if val_analysis:
    analysis_dict['val'] = val_analysis

if analysis_dict:
    plot_class_distribution(analysis_dict, FIGURES_DIR / 'class_distribution.png')

## Image Characteristics {#image-characteristics}

In [ ]:
def plot_image_characteristics(analysis_results, save_path=None):
    """Plot image size distribution and other characteristics."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    for split_name, analysis in analysis_results.items():
        if analysis is None:
            continue
        
        # Image sizes
        widths = [size[0] for size in analysis['image_sizes']]
        heights = [size[1] for size in analysis['image_sizes']]
        
        # Plot 1: Image width distribution
        axes[0, 0].hist(widths, bins=30, alpha=0.7, label=split_name)
        axes[0, 0].set_title('Image Width Distribution')
        axes[0, 0].set_xlabel('Width (pixels)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].legend()
        
        # Plot 2: Image height distribution
        axes[0, 1].hist(heights, bins=30, alpha=0.7, label=split_name)
        axes[0, 1].set_title('Image Height Distribution')
        axes[0, 1].set_xlabel('Height (pixels)')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].legend()
        
        # Plot 3: Annotations per image
        axes[1, 0].hist(analysis['annotations_per_image'], bins=30, alpha=0.7, label=split_name)
        axes[1, 0].set_title('Annotations per Image')
        axes[1, 0].set_xlabel('Number of Annotations')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].legend()
        
        # Plot 4: Bounding box areas
        log_areas = np.log10(np.array(analysis['bbox_areas']) + 1)
        axes[1, 1].hist(log_areas, bins=30, alpha=0.7, label=split_name)
        axes[1, 1].set_title('Bounding Box Areas (log scale)')
        axes[1, 1].set_xlabel('Log10(Area + 1)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

if analysis_dict:
    plot_image_characteristics(analysis_dict, FIGURES_DIR / 'image_characteristics.png')
    
    # Print statistics
    for split_name, analysis in analysis_dict.items():
        if analysis:
            print(f"\n{split_name.title()} Split Statistics:")
            print(f"  Total images: {analysis['total_images']}")
            print(f"  Total annotations: {sum(analysis['class_distribution'].values())}")
            print(f"  Avg annotations per image: {np.mean(analysis['annotations_per_image']):.1f}")
            print(f"  Avg bbox area: {np.mean(analysis['bbox_areas']):.1f} pixels²")

## Annotation Analysis {#annotation-analysis}

In [ ]:
def analyze_annotation_quality(analysis_results):
    """Analyze annotation quality metrics."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    for split_name, analysis in analysis_results.items():
        if analysis is None:
            continue
        
        # Truncation statistics
        truncation_labels = ['No truncation', 'Partial truncation', 'Heavy truncation']
        truncation_counts = [analysis['truncation_stats'].get(i, 0) for i in range(3)]
        
        axes[0].bar(truncation_labels, truncation_counts, alpha=0.7, label=split_name)
        axes[0].set_title('Truncation Statistics')
        axes[0].set_ylabel('Number of Objects')
        axes[0].legend()
        
        # Occlusion statistics
        occlusion_labels = ['No occlusion', 'Partial occlusion', 'Heavy occlusion']
        occlusion_counts = [analysis['occlusion_stats'].get(i, 0) for i in range(3)]
        
        axes[1].bar(occlusion_labels, occlusion_counts, alpha=0.7, label=split_name)
        axes[1].set_title('Occlusion Statistics')
        axes[1].set_ylabel('Number of Objects')
        axes[1].legend()
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'annotation_quality.png', dpi=300, bbox_inches='tight')
    plt.show()

if analysis_dict:
    analyze_annotation_quality(analysis_dict)

## Sample Images Visualization

In [ ]:
def visualize_sample_images(dataset_dir, split='train', num_samples=4):
    """Visualize sample images with annotations."""
    split_dir = dataset_dir / split
    if not split_dir.exists():
        print(f"Split {split} not found")
        return
    
    images_dir = split_dir / 'images'
    annotations_dir = split_dir / 'annotations'
    
    image_files = list(images_dir.glob('*.jpg'))[:num_samples]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(VISDRONE_CLASSES)))
    
    for i, img_file in enumerate(image_files):
        # Load image
        img = cv2.imread(str(img_file))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Load annotations
        ann_file = annotations_dir / (img_file.stem + '.txt')
        annotations = load_visdrone_annotations(ann_file)
        
        # Draw annotations
        for ann in annotations:
            if ann['object_category'] == 0:  # Skip ignored regions
                continue
            
            x1 = ann['bbox_left']
            y1 = ann['bbox_top']
            x2 = x1 + ann['bbox_width']
            y2 = y1 + ann['bbox_height']
            
            color = colors[ann['object_category'] % len(colors)][:3]
            color = tuple(int(c * 255) for c in color)
            
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color, 2)
            
            # Add class label
            class_name = VISDRONE_CLASSES.get(ann['object_category'], 'unknown')
            cv2.putText(img_rgb, class_name, (x1, y1-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # Display
        axes[i].imshow(img_rgb)
        axes[i].set_title(f'{img_file.name}\n{len(annotations)} objects')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / f'sample_images_{split}.png', dpi=300, bbox_inches='tight')
    plt.show()

# Visualize sample images
if train_analysis:
    visualize_sample_images(DATASET_DIR, 'train')
elif val_analysis:
    visualize_sample_images(DATASET_DIR, 'val')

## Challenges and Opportunities {#challenges}

Based on the dataset analysis, we can identify several key challenges and opportunities for autonomous drone detection:

### Challenges
1. **Small Objects**: Many objects in drone imagery are small, requiring specialized detection techniques
2. **Class Imbalance**: Some classes have significantly more instances than others
3. **Occlusion and Truncation**: Real-world conditions create challenging detection scenarios
4. **Scale Variation**: Objects appear at various scales due to altitude differences

### Opportunities
1. **Topological Filtering**: Betti numbers can help identify legitimate object groupings vs. false positives
2. **Contextual Reasoning**: Spatial relationships between objects can improve detection
3. **Multi-scale Training**: The dataset provides good coverage of different scales
4. **Real-world Applicability**: The dataset represents realistic drone deployment scenarios

In [ ]:
# Save analysis summary
summary = {
    'dataset_overview': {
        'classes': VISDRONE_CLASSES,
        'splits_analyzed': list(analysis_dict.keys())
    },
    'analysis_results': {}
}

for split_name, analysis in analysis_dict.items():
    if analysis:
        summary['analysis_results'][split_name] = {
            'total_images': analysis['total_images'],
            'total_annotations': sum(analysis['class_distribution'].values()),
            'avg_annotations_per_image': float(np.mean(analysis['annotations_per_image'])),
            'class_distribution': dict(analysis['class_distribution']),
            'avg_bbox_area': float(np.mean(analysis['bbox_areas'])) if analysis['bbox_areas'] else 0
        }

# Save to file
with open(FIGURES_DIR / 'dataset_analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("Dataset exploration completed!")
print(f"Results saved to: {FIGURES_DIR}")